In [13]:
import sys
sys.path.insert(1, '/home/eatitall_scripts')
sys.path.insert(1, '/home/root/pctobs/lib/python3.8/site-packages')
import numpy as np
import pandas as pd
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [4]:
# Cargamos los NOMBRES de los paths
data_path='./../../archivos/datos_con_10_ejemplos_reglas_y_extraccion_entidades.csv'

# Cargamos los datos
df = pd.read_csv(data_path)

In [19]:
lista_alimentos=[]
for element in df['alimentos_encontrados']:
    lista_alimentos.append(element)
lista_alimentos

['carbohidratos refinados, pan blanco, carbohidratos complejos, avena, quinoa, legumbres',
 'carbohidratos, carbohidratos',
 'agua, bebidas azucaradas',
 'grasas, aceite de oliva, aguacates, frutos secos, grasas',
 'carbohidratos',
 'bebidas azucaradas',
 'carbohidratos simples, vegetales',
 'snacks ricos en carbohidratos complejos',
 nan,
 'fruta fresca']

In [14]:
# Descargar recursos de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres no alfabéticos
    text = re.sub(r'[^a-z]', ' ', text)
    # Tokenización
    words = text.split()
    # Eliminación de stopwords
    words = [word for word in words if word not in stopwords.words('spanish')]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
documents = lista_alimentos[:8]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([preprocess_text(doc) for doc in documents])

from sklearn.cluster import DBSCAN

# Aplicar DBSCAN
dbscan = DBSCAN(eps=1.25, min_samples=2)
clusters = dbscan.fit_predict(X)


In [21]:
# Número de clusters (excluyendo el ruido)
n_clusters_ = len(set(clusters)) - (1 if -1 in clusters else 0)
n_noise_ = list(clusters).count(-1)

print(f'Número estimado de clusters: {n_clusters_}')
print(f'Número estimado de puntos de ruido: {n_noise_}')

# Ejemplo para imprimir los documentos de un cluster específico
for cluster_id in np.unique(clusters):
    if cluster_id == -1:
        # Saltar el ruido
        continue
    print(f"\nCluster {cluster_id}")
    for i, label in enumerate(clusters):
        if label == cluster_id:
            print(f" - {documents[i]}")

Número estimado de clusters: 2
Número estimado de puntos de ruido: 1

Cluster 0
 - carbohidratos refinados, pan blanco, carbohidratos complejos, avena, quinoa, legumbres
 - carbohidratos, carbohidratos
 - carbohidratos
 - carbohidratos simples, vegetales
 - snacks ricos en carbohidratos complejos

Cluster 1
 - agua, bebidas azucaradas
 - bebidas azucaradas
